### **Axial LOB**

In [1]:
#1 
#Load necessary packages
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from tqdm import tqdm 
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay
import torch
from torch.utils import data
import torch.nn as nn
import torch.optim as optim

# Set device (GPU if available)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cpu


In [2]:
#2 
#Import necessary packages
import numpy as np

# Define paths based on your directory structure for NoAuction
train_file = 'C:/Users/monam/BenchmarkDatasets/NoAuction/1.NoAuction_Zscore/NoAuction_Zscore_Training/Train_Dst_NoAuction_ZScore_CF_7.txt'
test_file1 = 'C:/Users/monam/BenchmarkDatasets/NoAuction/1.NoAuction_Zscore/NoAuction_Zscore_Testing/Test_Dst_NoAuction_ZScore_CF_7.txt'
test_file2 = 'C:/Users/monam/BenchmarkDatasets/NoAuction/1.NoAuction_Zscore/NoAuction_Zscore_Testing/Test_Dst_NoAuction_ZScore_CF_8.txt'
test_file3 = 'C:/Users/monam/BenchmarkDatasets/NoAuction/1.NoAuction_Zscore/NoAuction_Zscore_Testing/Test_Dst_NoAuction_ZScore_CF_9.txt'

# Load training and validation data
dec_data = np.loadtxt(train_file)
dec_train = dec_data[:, :int(dec_data.shape[1] * 0.8)]
dec_val = dec_data[:, int(dec_data.shape[1] * 0.8):]

# Load test data and concatenate
dec_test1 = np.loadtxt(test_file1)
dec_test2 = np.loadtxt(test_file2)
dec_test3 = np.loadtxt(test_file3)
dec_test = np.hstack((dec_test1, dec_test2, dec_test3))

# Set parameters
W = 40                     # Number of features
dim = 40                   # Number of LOB states
horizon = 2                # Horizon for target calculation
T = 5                      # Time window size for dataset creation

# Prepare labels
y_train = dec_train[-horizon, :].flatten()
y_val = dec_val[-horizon, :].flatten()
y_test = dec_test[-horizon, :].flatten()

# Adjust labels for training, validation, and test sets
y_train = y_train[dim-1:] - 1
y_val = y_val[dim-1:] - 1
y_test = y_test[dim-1:] - 1 

# Prepare data for model input
dec_train = dec_train[:40, :].T
dec_val = dec_val[:40, :].T
dec_test = dec_test[:40, :].T

# Print shapes to verify data
print("Training data shape:", dec_train.shape)
print("Validation data shape:", dec_val.shape)
print("Testing data shape:", dec_test.shape)


Training data shape: (203800, 40)
Validation data shape: (50950, 40)
Testing data shape: (139587, 40)


In [5]:
# 3
import torch
from torch.utils import data

class Dataset(data.Dataset):
    """Characterizes a dataset for PyTorch"""
    def __init__(self, x, y, num_classes, dim):
        """Initialization""" 
        self.num_classes = num_classes
        self.dim = dim
        self.x = x   
        self.y = y

        # Compute length based on rolling window
        self.length = x.shape[0] - T - self.dim + 1
        print("Dataset length:", self.length)

        # Convert to PyTorch tensors
        x = torch.from_numpy(x).float()  # Ensure data is float for model input
        self.x = torch.unsqueeze(x, 1)   # Add channel dimension
        self.y = torch.from_numpy(y).long()  # Labels should be long type for classification

    def __len__(self):
        """Denotes the total number of samples"""
        return self.length

    def __getitem__(self, i):
        # Extract input with rolling window and adjust shape
        input = self.x[i:i+self.dim, :]
        input = input.permute(1, 0, 2)  # Adjust to expected shape [1, dim, features]
        
        return input, self.y[i]

# Set parameters
batch_size = 64
num_classes = 3  # Adjust based on your problem (e.g., 3 classes for LOB levels)
dim = 40  # Number of LOB states, adjust as needed

# Instantiate Dataset objects for train, validation, and test sets
dataset_train = Dataset(dec_train, y_train, num_classes, dim)
dataset_val = Dataset(dec_val, y_val, num_classes, dim)
dataset_test = Dataset(dec_test, y_test, num_classes, dim)

# Create DataLoader objects for batching and shuffling
train_loader = data.DataLoader(dataset=dataset_train, batch_size=batch_size, shuffle=True)
val_loader = data.DataLoader(dataset=dataset_val, batch_size=batch_size, shuffle=False)
test_loader = data.DataLoader(dataset=dataset_test, batch_size=batch_size, shuffle=False)

# Verify DataLoader functionality
for inputs, labels in train_loader:
    print("Input batch shape:", inputs.shape)
    print("Label batch shape:", labels.shape)
    break  # Test with a single batch


Dataset length: 203756
Dataset length: 50906
Dataset length: 139543
Input batch shape: torch.Size([64, 1, 40, 40])
Label batch shape: torch.Size([64])


In [6]:
#4
# Import necessary packages
import torch
from torch.utils import data

# Define hyperparameters
batch_size = 64
epochs = 50 
c_final = 4              # Channel output size of the second conv layer
n_heads = 4
c_in_axial = 32          # Channel output size of the first conv layer
c_out_axial = 32
pool_kernel = (1, 4)
pool_stride = (1, 4)

num_classes = 3

# Adjust label preparation without flattening the entire dataset
horizon = 2
dim = 40

# Define lengths based on data size and required horizon offset
train_len = dec_train.shape[0] - dim + 1 - horizon
val_len = dec_val.shape[0] - dim + 1 - horizon
test_len = dec_test.shape[0] - dim + 1 - horizon

# Slicing the labels to match the data lengths exactly
y_train = dec_train[dim-1:dim-1 + train_len, -horizon] - 1
y_val = dec_val[dim-1:dim-1 + val_len, -horizon] - 1
y_test = dec_test[dim-1:dim-1 + test_len, -horizon] - 1

# Confirm alignment
print("Training data shape:", dec_train[:train_len].shape)
print("Training labels shape:", y_train.shape)
print("Validation data shape:", dec_val[:val_len].shape)
print("Validation labels shape:", y_val.shape)
print("Testing data shape:", dec_test[:test_len].shape)
print("Testing labels shape:", y_test.shape)

# Create Dataset instances
dataset_train = Dataset(dec_train, y_train, num_classes, dim)
dataset_val = Dataset(dec_val, y_val, num_classes, dim)
dataset_test = Dataset(dec_test, y_test, num_classes, dim)

# Set up DataLoader instances
train_loader = torch.utils.data.DataLoader(dataset=dataset_train, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(dataset=dataset_val, batch_size=batch_size, shuffle=False)
test_loader = torch.utils.data.DataLoader(dataset=dataset_test, batch_size=batch_size, shuffle=False)

# Verify DataLoader functionality with a sample batch
for inputs, labels in train_loader:
    print("Sample input batch shape:", inputs.shape)
    print("Sample label batch shape:", labels.shape)
    break  # Test with a single batch


Training data shape: (203759, 40)
Training labels shape: (203759,)
Validation data shape: (50909, 40)
Validation labels shape: (50909,)
Testing data shape: (139546, 40)
Testing labels shape: (139546,)
Dataset length: 203756
Dataset length: 50906
Dataset length: 139543
Sample input batch shape: torch.Size([64, 1, 40, 40])
Sample label batch shape: torch.Size([64])


In [7]:
#5
# Import necessary packages
import torch
from torch.utils import data
import torch.nn as nn
import math

# Define hyperparameters
batch_size = 64
epochs = 50 
c_final = 4
n_heads = 4
c_in_axial = 32
c_out_axial = 32
pool_kernel = (1, 4)
pool_stride = (1, 4)
num_classes = 3

# Ensure labels align with data by slicing both consistently
horizon = 2
dim = 40
train_len = dec_train.shape[0] - dim + 1 - horizon
val_len = dec_val.shape[0] - dim + 1 - horizon
test_len = dec_test.shape[0] - dim + 1 - horizon
y_train = dec_train[dim-1:dim-1 + train_len, -horizon] - 1
y_val = dec_val[dim-1:dim-1 + val_len, -horizon] - 1
y_test = dec_test[dim-1:dim-1 + test_len, -horizon] - 1

# Dataset setup
dataset_train = Dataset(dec_train[:train_len], y_train, num_classes, dim)
dataset_val = Dataset(dec_val[:val_len], y_val, num_classes, dim)
dataset_test = Dataset(dec_test[:test_len], y_test, num_classes, dim)
train_loader = torch.utils.data.DataLoader(dataset=dataset_train, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(dataset=dataset_val, batch_size=batch_size, shuffle=False)
test_loader = torch.utils.data.DataLoader(dataset=dataset_test, batch_size=batch_size, shuffle=False)

# Model Architecture
def _conv1d1x1(in_channels, out_channels):
    return nn.Sequential(
        nn.Conv1d(in_channels, out_channels, kernel_size=1, stride=1, bias=False),
        nn.BatchNorm1d(out_channels)
    )

class GatedAxialAttention(nn.Module):
    def __init__(self, in_channels, out_channels, heads, dim, flag):
        super().__init__()
        assert (in_channels % heads == 0) and (out_channels % heads == 0)
        self.in_channels, self.out_channels, self.heads = in_channels, out_channels, heads
        self.dim_head_v = out_channels // heads
        self.flag, self.dim = flag, dim
        self.dim_head_qk = self.dim_head_v // 2
        self.qkv_channels = self.dim_head_v + self.dim_head_qk * 2

        self.to_qkv = _conv1d1x1(in_channels, heads * self.qkv_channels)
        self.bn_qkv, self.bn_similarity = nn.BatchNorm1d(heads * self.qkv_channels), nn.BatchNorm2d(heads * 3)
        self.bn_output = nn.BatchNorm1d(heads * self.qkv_channels)

        # Gating mechanism
        self.f_qr, self.f_kr = nn.Parameter(torch.tensor(0.3), False), nn.Parameter(torch.tensor(0.3), False)
        self.f_sve, self.f_sv = nn.Parameter(torch.tensor(0.3), False), nn.Parameter(torch.tensor(0.5), False)

        # Position embedding
        self.relative = nn.Parameter(torch.randn(self.dim_head_v * 2, dim * 2 - 1), requires_grad=True)
        query_index = torch.arange(dim).unsqueeze(0)
        key_index = torch.arange(dim).unsqueeze(1)
        relative_index = key_index - query_index + dim - 1
        self.register_buffer('flatten_index', relative_index.view(-1))

    def forward(self, x):
        if self.flag:
            x = x.permute(0, 2, 1, 3)
        else:
            x = x.permute(0, 3, 1, 2)  # N, W, C, H
        N, W, C, H = x.shape
        x = x.view(N * W, C, H)

        x = self.to_qkv(x)
        qkv = self.bn_qkv(x)
        q, k, v = torch.split(qkv.reshape(N * W, self.heads, self.dim_head_v * 2, H),
                              [self.dim_head_v // 2, self.dim_head_v // 2, self.dim_head_v], dim=2)

        all_embeddings = torch.index_select(self.relative, 1, self.flatten_index).view(self.dim_head_v * 2, self.dim, self.dim)
        q_embedding, k_embedding, v_embedding = torch.split(all_embeddings, [self.dim_head_qk, self.dim_head_qk, self.dim_head_v], dim=0)
        qr, kr, qk = torch.einsum('bgci,cij->bgij', q, q_embedding), torch.einsum('bgci,cij->bgij', k, k_embedding).transpose(2, 3), torch.einsum('bgci, bgcj->bgij', q, k)
        qr, kr = torch.mul(qr, self.f_qr), torch.mul(kr, self.f_kr)

        similarity = torch.softmax(self.bn_similarity(torch.cat([qk, qr, kr], dim=1)).view(N * W, 3, self.heads, H, H).sum(dim=1), dim=3)
        sv, sve = torch.mul(torch.einsum('bgij,bgcj->bgci', similarity, v), self.f_sv), torch.mul(torch.einsum('bgij,cij->bgci', similarity, v_embedding), self.f_sve)

        output = self.bn_output(torch.cat([sv, sve], dim=-1).view(N * W, self.out_channels * 2, H)).view(N, W, self.out_channels, 2, H).sum(dim=-2)
        return output.permute(0, 2, 3, 1) if not self.flag else output.permute(0, 2, 1, 3)

class AxialLOB(nn.Module):
    def __init__(self, W, H, c_in, c_out, c_final, n_heads, pool_kernel, pool_stride):
        super().__init__()
        self.CNN_in, self.CNN_out = nn.Conv2d(1, c_in, kernel_size=1), nn.Conv2d(c_out, c_final, kernel_size=1)
        self.CNN_res2, self.CNN_res1 = nn.Conv2d(c_out, c_final, kernel_size=1), nn.Conv2d(1, c_out, kernel_size=1)
        self.norm, self.res_norm2, self.res_norm1, self.norm2 = nn.BatchNorm2d(c_in), nn.BatchNorm2d(c_final), nn.BatchNorm2d(c_out), nn.BatchNorm2d(c_final)
        self.axial_height_1, self.axial_width_1 = GatedAxialAttention(c_out, c_out, n_heads, H, False), GatedAxialAttention(c_out, c_out, n_heads, W, True)
        self.axial_height_2, self.axial_width_2 = GatedAxialAttention(c_out, c_out, n_heads, H, False), GatedAxialAttention(c_out, c_out, n_heads, W, True)
        self.activation, self.linear = nn.ReLU(), nn.Linear(1600, 3)
        self.pooling = nn.AvgPool2d(kernel_size=pool_kernel, stride=pool_stride)

    def forward(self, x):
        y = self.activation(self.norm(self.CNN_in(x)))
        y, x = self.axial_width_1(y), self.activation(self.res_norm1(self.CNN_res1(x)))
        y, y_copy = y + x, y.detach().clone()
        y = self.axial_width_2(y + x)
        y = self.activation(self.res_norm2(self.CNN_out(self.axial_height_2(self.axial_height_1(y)))))
        return torch.softmax(self.linear(torch.flatten(self.pooling(y + self.activation(self.norm2(self.CNN_res2(y_copy)))), 1)), dim=1)


Dataset length: 203715
Dataset length: 50865
Dataset length: 139502


In [22]:
#6
# Import necessary packages
import os
import numpy as np
import torch
from torch.utils import data
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score
import pandas as pd

# Set device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# File paths
train_file = 'C:/Users/monam/BenchmarkDatasets/NoAuction/1.NoAuction_Zscore/NoAuction_Zscore_Training/Train_Dst_NoAuction_ZScore_CF_7.txt'
test_file1 = 'C:/Users/monam/BenchmarkDatasets/NoAuction/1.NoAuction_Zscore/NoAuction_Zscore_Testing/Test_Dst_NoAuction_ZScore_CF_7.txt'
test_file2 = 'C:/Users/monam/BenchmarkDatasets/NoAuction/1.NoAuction_Zscore/NoAuction_Zscore_Testing/Test_Dst_NoAuction_ZScore_CF_8.txt'
test_file3 = 'C:/Users/monam/BenchmarkDatasets/NoAuction/1.NoAuction_Zscore/NoAuction_Zscore_Testing/Test_Dst_NoAuction_ZScore_CF_9.txt'

# Function to load and preprocess data
def load_data(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    # Clean lines: Remove extra spaces and ensure proper formatting
    cleaned_lines = [line.strip().replace('   ', ' ').replace('  ', ' ') for line in lines]
    # Load as NumPy array
    data = np.genfromtxt(cleaned_lines, delimiter=' ')
    return data

# Load datasets
dec_train = load_data(train_file)
dec_test1 = load_data(test_file1)
dec_test2 = load_data(test_file2)
dec_test3 = load_data(test_file3)

print("Training data shape:", dec_train.shape)

# Ensure the test data is padded consistently
def pad_array(arr, target_cols):
    padding = target_cols - arr.shape[1]
    return np.pad(arr, ((0, 0), (0, padding)), mode='constant')

def pad_arrays(*arrays):
    max_cols = max(arr.shape[1] for arr in arrays)
    return [pad_array(arr, max_cols) for arr in arrays]

dec_test1, dec_test2, dec_test3 = pad_arrays(dec_test1, dec_test2, dec_test3)
test_data_combined = np.vstack((dec_test1, dec_test2, dec_test3))

# Parameters
W, dim, num_classes, batch_size = 40, 40, 3, 32

# Create sequences
def create_sequences(data, dim, horizon, W, num_classes):
    sequences, labels = [], []
    for i in range(data.shape[0] - dim - horizon + 1):
        seq = data[i:i + dim, :W]
        label = int(data[i + dim + horizon - 1, -1])
        if 0 <= label < num_classes:
            sequences.append(seq)
            labels.append(label)
    return np.array(sequences), np.array(labels, dtype=np.int64)

# Dataset class
class Dataset(data.Dataset):
    def __init__(self, X, y):
        self.X, self.y = X, y

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        X = torch.tensor(self.X[idx], dtype=torch.float32).unsqueeze(0)
        y = torch.tensor(self.y[idx], dtype=torch.long)
        return X, y

# Test with AxialLOB model
class AxialLOB(nn.Module):
    def __init__(self, W, H, c_in, c_out, c_final, num_classes):
        super().__init__()
        self.conv1 = nn.Conv2d(1, c_in, kernel_size=1)
        self.conv2 = nn.Conv2d(c_in, c_out, kernel_size=1)
        self.conv3 = nn.Conv2d(c_out, c_final, kernel_size=1)
        self.pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(c_final, num_classes)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.relu(self.conv2(x))
        x = torch.relu(self.conv3(x))
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        return self.fc(x)

# Instantiate the model
model = AxialLOB(W, dim, 16, 16, 16, num_classes).to(device)

# Print test data shape for debugging
print("Test data combined shape:", test_data_combined.shape)


Using device: cpu
Training data shape: (149, 254750)
Test data combined shape: (447, 55478)


In [29]:
#7 CNN, Axial LOB and Deep LOB - F1 scores
import os
import numpy as np
import torch
from torch.utils import data
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from sklearn.metrics import precision_score, recall_score, f1_score
import pandas as pd

# Set device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# File paths
train_file = 'C:/Users/monam/BenchmarkDatasets/NoAuction/1.NoAuction_Zscore/NoAuction_Zscore_Training/Train_Dst_NoAuction_ZScore_CF_7.txt'
test_file1 = 'C:/Users/monam/BenchmarkDatasets/NoAuction/1.NoAuction_Zscore/NoAuction_Zscore_Testing/Test_Dst_NoAuction_ZScore_CF_7.txt'
test_file2 = 'C:/Users/monam/BenchmarkDatasets/NoAuction/1.NoAuction_Zscore/NoAuction_Zscore_Testing/Test_Dst_NoAuction_ZScore_CF_8.txt'
test_file3 = 'C:/Users/monam/BenchmarkDatasets/NoAuction/1.NoAuction_Zscore/NoAuction_Zscore_Testing/Test_Dst_NoAuction_ZScore_CF_9.txt'

# Load data
def load_data(file_path):
    return np.loadtxt(file_path)

dec_train = load_data(train_file)
dec_test1 = load_data(test_file1)
dec_test2 = load_data(test_file2)
dec_test3 = load_data(test_file3)

# Preprocess to handle mismatched dimensions
def preprocess_data(*arrays):
    max_cols = max(array.shape[1] for array in arrays)
    processed_arrays = []
    for array in arrays:
        if array.shape[1] < max_cols:
            padded = np.pad(array, ((0, 0), (0, max_cols - array.shape[1])), mode='constant', constant_values=0)
            processed_arrays.append(padded)
        elif array.shape[1] > max_cols:
            processed_arrays.append(array[:, :max_cols])
        else:
            processed_arrays.append(array)
    return processed_arrays

dec_test1, dec_test2, dec_test3 = preprocess_data(dec_test1, dec_test2, dec_test3)

# Combine test data
test_data_combined = np.vstack([dec_test1, dec_test2, dec_test3])

# Parameters
W, dim, num_classes, batch_size = 40, 40, 3, 32

# Create sequences
def create_sequences(data, dim, horizon, W, num_classes):
    sequences, labels = [], []
    for i in range(data.shape[0] - dim - horizon + 1):
        seq = data[i:i + dim, :W]
        label = int(data[i + dim + horizon - 1, -1])
        if 0 <= label < num_classes:
            sequences.append(seq)
            labels.append(label)
    return np.array(sequences), np.array(labels, dtype=np.int64)

# Dataset class
class Dataset(data.Dataset):
    def __init__(self, X, y):
        self.X, self.y = X, y

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        X = torch.tensor(self.X[idx], dtype=torch.float32).unsqueeze(0)
        y = torch.tensor(self.y[idx], dtype=torch.long)
        return X, y

# Models
class CNN(nn.Module):
    def __init__(self, W, num_classes):
        super().__init__()
        self.conv = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1)
        self.pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(16, num_classes)

    def forward(self, x):
        x = torch.relu(self.conv(x))
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        return self.fc(x)

class AxialLOB(nn.Module):
    def __init__(self, W, H, c_in, c_out, c_final, num_classes):
        super().__init__()
        self.conv1 = nn.Conv2d(1, c_in, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(c_in)
        self.conv2 = nn.Conv2d(c_in, c_out, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(c_out)
        self.conv3 = nn.Conv2d(c_out, c_final, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(c_final)
        self.pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Sequential(
            nn.Linear(c_final, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        x = torch.relu(self.bn1(self.conv1(x)))
        x = torch.relu(self.bn2(self.conv2(x)))
        x = torch.relu(self.bn3(self.conv3(x)))
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        return self.fc(x)

class DeepLOB(nn.Module):
    def __init__(self, W, hidden_size, num_classes):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.lstm = nn.LSTM(W * 128, hidden_size, batch_first=True, bidirectional=True)
        self.fc = nn.Sequential(
            nn.Linear(2 * hidden_size, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        x = torch.relu(self.bn1(self.conv1(x)))
        x = torch.relu(self.bn2(self.conv2(x)))
        x = torch.relu(self.bn3(self.conv3(x)))
        batch_size, channels, height, width = x.size()
        x = x.view(batch_size, height, -1)
        x, _ = self.lstm(x)
        x = x[:, -1, :]
        return self.fc(x)

# Evaluation
def evaluate_all(models, train_data, test_data, horizons):
    results = []
    for model_name, model in models.items():
        print(f"Evaluating {model_name}")
        model.to(device)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=0.001)
        for horizon in horizons:
            print(f"Evaluating for Prediction Horizon: {horizon}")
            X_train, y_train = create_sequences(train_data, dim, horizon, W, num_classes)
            X_test, y_test = create_sequences(test_data, dim, horizon, W, num_classes)

            train_loader = data.DataLoader(Dataset(X_train, y_train), batch_size=batch_size, shuffle=True)
            test_loader = data.DataLoader(Dataset(X_test, y_test), batch_size=batch_size, shuffle=False)

            # Train
            for epoch in range(5):
                model.train()
                for inputs, labels in train_loader:
                    inputs, labels = inputs.to(device), labels.to(device)
                    optimizer.zero_grad()
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    loss.backward()
                    optimizer.step()

            # Evaluate
            model.eval()
            all_preds, all_labels = [], []
            with torch.no_grad():
                for inputs, labels in test_loader:
                    inputs, labels = inputs.to(device), labels.to(device)
                    outputs = model(inputs)
                    preds = torch.argmax(outputs, dim=1)
                    all_preds.append(preds.cpu().numpy())
                    all_labels.append(labels.cpu().numpy())

            preds = np.concatenate(all_preds)
            labels = np.concatenate(all_labels)
            precision = precision_score(labels, preds, average='macro', zero_division=0) * 100
            recall = recall_score(labels, preds, average='macro', zero_division=0) * 100
            f1 = f1_score(labels, preds, average='macro', zero_division=0) * 100

            results.append([model_name, f"Prediction Horizon = {horizon}", precision, recall, f1])

    return results

# Instantiate models
models = {
    "CNN": CNN(W, num_classes),
    "AxialLOB": AxialLOB(W, dim, 16, 16, 16, num_classes),
    "DeepLOB": DeepLOB(W, 64, num_classes),
}

# Evaluate
results = evaluate_all(models, dec_train, test_data_combined, [10, 20, 30, 50, 100])
df_results = pd.DataFrame(results, columns=["Model", "Prediction Horizon", "Precision (%)", "Recall (%)", "F1 (%)"])
print(df_results)


Using device: cpu
Evaluating CNN
Evaluating for Prediction Horizon: 10
Evaluating for Prediction Horizon: 20
Evaluating for Prediction Horizon: 30
Evaluating for Prediction Horizon: 50
Evaluating for Prediction Horizon: 100
Evaluating AxialLOB
Evaluating for Prediction Horizon: 10
Evaluating for Prediction Horizon: 20
Evaluating for Prediction Horizon: 30
Evaluating for Prediction Horizon: 50
Evaluating for Prediction Horizon: 100
Evaluating DeepLOB
Evaluating for Prediction Horizon: 10
Evaluating for Prediction Horizon: 20
Evaluating for Prediction Horizon: 30
Evaluating for Prediction Horizon: 50
Evaluating for Prediction Horizon: 100
       Model        Prediction Horizon  Precision (%)  Recall (%)     F1 (%)
0        CNN   Prediction Horizon = 10      31.476793   33.333333  32.378472
1        CNN   Prediction Horizon = 20      32.207792   33.333333  32.760898
2        CNN   Prediction Horizon = 30      32.355556   33.333333  32.837167
3        CNN   Prediction Horizon = 50      32.

In [30]:
# 8 B (tabl) and C (tabl) - F1 scores 
import os
import numpy as np
import torch
from torch.utils import data
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from sklearn.metrics import precision_score, recall_score, f1_score
import pandas as pd

# Set device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# File paths
train_file = 'C:/Users/monam/BenchmarkDatasets/NoAuction/1.NoAuction_Zscore/NoAuction_Zscore_Training/Train_Dst_NoAuction_ZScore_CF_7.txt'
test_file1 = 'C:/Users/monam/BenchmarkDatasets/NoAuction/1.NoAuction_Zscore/NoAuction_Zscore_Testing/Test_Dst_NoAuction_ZScore_CF_7.txt'
test_file2 = 'C:/Users/monam/BenchmarkDatasets/NoAuction/1.NoAuction_Zscore/NoAuction_Zscore_Testing/Test_Dst_NoAuction_ZScore_CF_8.txt'
test_file3 = 'C:/Users/monam/BenchmarkDatasets/NoAuction/1.NoAuction_Zscore/NoAuction_Zscore_Testing/Test_Dst_NoAuction_ZScore_CF_9.txt'

# Load data
def load_data(file_path):
    return np.loadtxt(file_path)

dec_train = load_data(train_file)
dec_test1 = load_data(test_file1)
dec_test2 = load_data(test_file2)
dec_test3 = load_data(test_file3)

# Preprocess to handle mismatched dimensions
def preprocess_data(*arrays):
    max_cols = max(array.shape[1] for array in arrays)
    processed_arrays = []
    for array in arrays:
        if array.shape[1] < max_cols:
            padded = np.pad(array, ((0, 0), (0, max_cols - array.shape[1])), mode='constant', constant_values=0)
            processed_arrays.append(padded)
        elif array.shape[1] > max_cols:
            processed_arrays.append(array[:, :max_cols])
        else:
            processed_arrays.append(array)
    return processed_arrays

dec_test1, dec_test2, dec_test3 = preprocess_data(dec_test1, dec_test2, dec_test3)

# Combine test data
test_data_combined = np.vstack([dec_test1, dec_test2, dec_test3])

# Parameters
W, dim, num_classes, batch_size = 40, 40, 3, 32

# Create sequences
def create_sequences(data, dim, horizon, W, num_classes):
    sequences, labels = [], []
    for i in range(data.shape[0] - dim - horizon + 1):
        seq = data[i:i + dim, :W]
        label = int(data[i + dim + horizon - 1, -1])
        if 0 <= label < num_classes:
            sequences.append(seq)
            labels.append(label)
    return np.array(sequences), np.array(labels, dtype=np.int64)

# Dataset class
class Dataset(data.Dataset):
    def __init__(self, X, y):
        self.X, self.y = X, y

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        X = torch.tensor(self.X[idx], dtype=torch.float32).unsqueeze(0)
        y = torch.tensor(self.y[idx], dtype=torch.long)
        return X, y

# Models
class B_TABL(nn.Module):
    def __init__(self, W, num_classes):
        super().__init__()
        self.fc1 = nn.Linear(W * W, 128)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = x.view(x.size(0), -1)  # Flatten input
        x = torch.relu(self.fc1(x))
        return self.fc2(x)

class C_TABL(nn.Module):
    def __init__(self, W, num_classes):
        super().__init__()
        self.fc1 = nn.Linear(W * W, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = x.view(x.size(0), -1)  # Flatten input
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return self.fc3(x)

# Evaluation
def evaluate_all(models, train_data, test_data, horizons):
    results = []
    for model_name, model in models.items():
        print(f"Evaluating {model_name}")
        model.to(device)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=0.001)
        for horizon in horizons:
            print(f"Evaluating for Prediction Horizon: {horizon}")
            X_train, y_train = create_sequences(train_data, dim, horizon, W, num_classes)
            X_test, y_test = create_sequences(test_data, dim, horizon, W, num_classes)

            train_loader = data.DataLoader(Dataset(X_train, y_train), batch_size=batch_size, shuffle=True)
            test_loader = data.DataLoader(Dataset(X_test, y_test), batch_size=batch_size, shuffle=False)

            # Train
            for epoch in range(5):
                model.train()
                for inputs, labels in train_loader:
                    inputs, labels = inputs.to(device), labels.to(device)
                    optimizer.zero_grad()
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    loss.backward()
                    optimizer.step()

            # Evaluate
            model.eval()
            all_preds, all_labels = [], []
            with torch.no_grad():
                for inputs, labels in test_loader:
                    inputs, labels = inputs.to(device), labels.to(device)
                    outputs = model(inputs)
                    preds = torch.argmax(outputs, dim=1)
                    all_preds.append(preds.cpu().numpy())
                    all_labels.append(labels.cpu().numpy())

            preds = np.concatenate(all_preds)
            labels = np.concatenate(all_labels)
            precision = precision_score(labels, preds, average='macro', zero_division=0) * 100
            recall = recall_score(labels, preds, average='macro', zero_division=0) * 100
            f1 = f1_score(labels, preds, average='macro', zero_division=0) * 100

            results.append([model_name, f"Prediction Horizon = {horizon}", precision, recall, f1])

    return results

# Instantiate models
models = {
    "B_TABL": B_TABL(W, num_classes),
    "C_TABL": C_TABL(W, num_classes),
}

# Evaluate
results = evaluate_all(models, dec_train, test_data_combined, [10, 20, 30, 50, 100])
df_results = pd.DataFrame(results, columns=["Model", "Prediction Horizon", "Precision (%)", "Recall (%)", "F1 (%)"])
print(df_results)


Using device: cpu
Evaluating B_TABL
Evaluating for Prediction Horizon: 10
Evaluating for Prediction Horizon: 20
Evaluating for Prediction Horizon: 30
Evaluating for Prediction Horizon: 50
Evaluating for Prediction Horizon: 100
Evaluating C_TABL
Evaluating for Prediction Horizon: 10
Evaluating for Prediction Horizon: 20
Evaluating for Prediction Horizon: 30
Evaluating for Prediction Horizon: 50
Evaluating for Prediction Horizon: 100
    Model        Prediction Horizon  Precision (%)  Recall (%)     F1 (%)
0  B_TABL   Prediction Horizon = 10      37.788537   55.406613  34.849938
1  B_TABL   Prediction Horizon = 20      33.338251   41.829586  28.227732
2  B_TABL   Prediction Horizon = 30      33.931624   31.359381  32.081888
3  B_TABL   Prediction Horizon = 50      32.671082   28.271251  30.312340
4  B_TABL  Prediction Horizon = 100      35.111111   55.445545  29.962829
5  C_TABL   Prediction Horizon = 10      36.345008   43.923146  36.271320
6  C_TABL   Prediction Horizon = 20      32.16

In [31]:
# 9 Deep LOB seq2seq and deep LOB attention- F1 scores
import os
import numpy as np
import torch
from torch.utils import data
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from sklearn.metrics import precision_score, recall_score, f1_score
import pandas as pd

# Set device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# File paths
train_file = 'C:/Users/monam/BenchmarkDatasets/NoAuction/1.NoAuction_Zscore/NoAuction_Zscore_Training/Train_Dst_NoAuction_ZScore_CF_7.txt'
test_file1 = 'C:/Users/monam/BenchmarkDatasets/NoAuction/1.NoAuction_Zscore/NoAuction_Zscore_Testing/Test_Dst_NoAuction_ZScore_CF_7.txt'
test_file2 = 'C:/Users/monam/BenchmarkDatasets/NoAuction/1.NoAuction_Zscore/NoAuction_Zscore_Testing/Test_Dst_NoAuction_ZScore_CF_8.txt'
test_file3 = 'C:/Users/monam/BenchmarkDatasets/NoAuction/1.NoAuction_Zscore/NoAuction_Zscore_Testing/Test_Dst_NoAuction_ZScore_CF_9.txt'

# Load data
def load_data(file_path):
    return np.loadtxt(file_path)

dec_train = load_data(train_file)
dec_test1 = load_data(test_file1)
dec_test2 = load_data(test_file2)
dec_test3 = load_data(test_file3)

# Preprocess to handle mismatched dimensions
def preprocess_data(*arrays):
    max_cols = max(array.shape[1] for array in arrays)
    processed_arrays = []
    for array in arrays:
        if array.shape[1] < max_cols:
            padded = np.pad(array, ((0, 0), (0, max_cols - array.shape[1])), mode='constant', constant_values=0)
            processed_arrays.append(padded)
        elif array.shape[1] > max_cols:
            processed_arrays.append(array[:, :max_cols])
        else:
            processed_arrays.append(array)
    return processed_arrays

dec_test1, dec_test2, dec_test3 = preprocess_data(dec_test1, dec_test2, dec_test3)

# Combine test data
test_data_combined = np.vstack([dec_test1, dec_test2, dec_test3])

# Parameters
W, dim, num_classes, batch_size = 40, 40, 3, 32

# Create sequences
def create_sequences(data, dim, horizon, W, num_classes):
    sequences, labels = [], []
    for i in range(data.shape[0] - dim - horizon + 1):
        seq = data[i:i + dim, :W]
        label = int(data[i + dim + horizon - 1, -1])
        if 0 <= label < num_classes:
            sequences.append(seq)
            labels.append(label)
    return np.array(sequences), np.array(labels, dtype=np.int64)

# Dataset class
class Dataset(data.Dataset):
    def __init__(self, X, y):
        self.X, self.y = X, y

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        X = torch.tensor(self.X[idx], dtype=torch.float32).unsqueeze(0)
        y = torch.tensor(self.y[idx], dtype=torch.long)
        return X, y

# Models
class DeepLOBSeq2Seq(nn.Module):
    def __init__(self, W, hidden_size, num_classes):
        super().__init__()
        self.encoder = nn.LSTM(W, hidden_size, batch_first=True, bidirectional=True)
        self.decoder = nn.LSTM(W, hidden_size, batch_first=True, bidirectional=True)
        self.fc = nn.Sequential(
            nn.Linear(2 * hidden_size, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        batch_size, _, height, _ = x.size()
        x = x.view(batch_size, height, -1)  # Flatten spatial dimensions
        _, (hidden, _) = self.encoder(x)  # Encoder outputs
        x, _ = self.decoder(x)  # Decoder processes input
        x = x[:, -1, :]  # Use last time-step
        return self.fc(x)

class DeepLOBAttention(nn.Module):
    def __init__(self, W, hidden_size, num_classes):
        super().__init__()
        self.encoder = nn.LSTM(W, hidden_size, batch_first=True, bidirectional=True)
        self.attention = nn.Sequential(
            nn.Linear(2 * hidden_size, 128),
            nn.Tanh(),
            nn.Linear(128, 1),
            nn.Softmax(dim=1)
        )
        self.fc = nn.Sequential(
            nn.Linear(2 * hidden_size, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        batch_size, _, height, _ = x.size()
        x = x.view(batch_size, height, -1)  # Flatten spatial dimensions
        x, _ = self.encoder(x)  # Encoder outputs
        attention_weights = self.attention(x)  # Compute attention weights
        x = torch.sum(attention_weights * x, dim=1)  # Apply attention
        return self.fc(x)

# Evaluation
def evaluate_all(models, train_data, test_data, horizons):
    results = []
    for model_name, model in models.items():
        print(f"Evaluating {model_name}")
        model.to(device)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=0.001)
        for horizon in horizons:
            print(f"Evaluating for Prediction Horizon: {horizon}")
            X_train, y_train = create_sequences(train_data, dim, horizon, W, num_classes)
            X_test, y_test = create_sequences(test_data, dim, horizon, W, num_classes)

            train_loader = data.DataLoader(Dataset(X_train, y_train), batch_size=batch_size, shuffle=True)
            test_loader = data.DataLoader(Dataset(X_test, y_test), batch_size=batch_size, shuffle=False)

            # Train
            for epoch in range(5):
                model.train()
                for inputs, labels in train_loader:
                    inputs, labels = inputs.to(device), labels.to(device)
                    optimizer.zero_grad()
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    loss.backward()
                    optimizer.step()

            # Evaluate
            model.eval()
            all_preds, all_labels = [], []
            with torch.no_grad():
                for inputs, labels in test_loader:
                    inputs, labels = inputs.to(device), labels.to(device)
                    outputs = model(inputs)
                    preds = torch.argmax(outputs, dim=1)
                    all_preds.append(preds.cpu().numpy())
                    all_labels.append(labels.cpu().numpy())

            preds = np.concatenate(all_preds)
            labels = np.concatenate(all_labels)
            precision = precision_score(labels, preds, average='macro', zero_division=0) * 100
            recall = recall_score(labels, preds, average='macro', zero_division=0) * 100
            f1 = f1_score(labels, preds, average='macro', zero_division=0) * 100

            results.append([model_name, f"Prediction Horizon = {horizon}", precision, recall, f1])

    return results

# Instantiate models
models = {
    "DeepLOB-Seq2Seq": DeepLOBSeq2Seq(W, 64, num_classes),
    "DeepLOB-Attention": DeepLOBAttention(W, 64, num_classes),
}

# Evaluate
results = evaluate_all(models, dec_train, test_data_combined, [10, 20, 30, 50, 100])
df_results = pd.DataFrame(results, columns=["Model", "Prediction Horizon", "Precision (%)", "Recall (%)", "F1 (%)"])
print(df_results)


Using device: cpu
Evaluating DeepLOB-Seq2Seq
Evaluating for Prediction Horizon: 10
Evaluating for Prediction Horizon: 20
Evaluating for Prediction Horizon: 30
Evaluating for Prediction Horizon: 50
Evaluating for Prediction Horizon: 100
Evaluating DeepLOB-Attention
Evaluating for Prediction Horizon: 10
Evaluating for Prediction Horizon: 20
Evaluating for Prediction Horizon: 30
Evaluating for Prediction Horizon: 50
Evaluating for Prediction Horizon: 100
               Model        Prediction Horizon  Precision (%)  Recall (%)  \
0    DeepLOB-Seq2Seq   Prediction Horizon = 10      31.476793   33.333333   
1    DeepLOB-Seq2Seq   Prediction Horizon = 20      32.207792   33.333333   
2    DeepLOB-Seq2Seq   Prediction Horizon = 30      32.355556   33.333333   
3    DeepLOB-Seq2Seq   Prediction Horizon = 50      32.677903   33.333333   
4    DeepLOB-Seq2Seq  Prediction Horizon = 100      32.792208   33.333333   
5  DeepLOB-Attention   Prediction Horizon = 10      31.467345   33.154602   
6  De

In [36]:
# 10 Back testing on F1 dataset
import os
import numpy as np
import torch
from torch.utils import data
import torch.nn as nn
import torch.optim as optim
import pandas as pd

# Set device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# File paths
train_file = 'C:/Users/monam/BenchmarkDatasets/NoAuction/1.NoAuction_Zscore/NoAuction_Zscore_Training/Train_Dst_NoAuction_ZScore_CF_7.txt'
test_file1 = 'C:/Users/monam/BenchmarkDatasets/NoAuction/1.NoAuction_Zscore/NoAuction_Zscore_Testing/Test_Dst_NoAuction_ZScore_CF_7.txt'
test_file2 = 'C:/Users/monam/BenchmarkDatasets/NoAuction/1.NoAuction_Zscore/NoAuction_Zscore_Testing/Test_Dst_NoAuction_ZScore_CF_8.txt'
test_file3 = 'C:/Users/monam/BenchmarkDatasets/NoAuction/1.NoAuction_Zscore/NoAuction_Zscore_Testing/Test_Dst_NoAuction_ZScore_CF_9.txt'

# Load data
def load_data(file_path):
    return np.loadtxt(file_path)

dec_train = load_data(train_file)
dec_test1 = load_data(test_file1)
dec_test2 = load_data(test_file2)
dec_test3 = load_data(test_file3)

# Preprocess to handle mismatched dimensions
def preprocess_data(*arrays):
    max_cols = max(array.shape[1] for array in arrays)
    processed_arrays = []
    for array in arrays:
        if array.shape[1] < max_cols:
            padded = np.pad(array, ((0, 0), (0, max_cols - array.shape[1])), mode='constant', constant_values=0)
            processed_arrays.append(padded)
        elif array.shape[1] > max_cols:
            processed_arrays.append(array[:, :max_cols])
        else:
            processed_arrays.append(array)
    return processed_arrays

dec_train, dec_test1, dec_test2, dec_test3 = preprocess_data(dec_train, dec_test1, dec_test2, dec_test3)

# Combine test data
test_data_combined = np.vstack([dec_test1, dec_test2, dec_test3])

# Parameters
W, dim, num_classes, batch_size = 40, 40, 3, 32

# Create sequences
def create_sequences(data, dim, horizon, W, num_classes):
    sequences, labels, prices = [], [], []
    N = data.shape[0]
    for i in range(N - dim - horizon + 1):
        seq = data[i:i + dim, :W]
        label = int(data[i + dim + horizon - 1, -1])  # Labels: 0 (Sell), 1 (Hold), 2 (Buy)
        price = data[i + dim + horizon - 1, 0]  # Assuming first column is price
        if 0 <= label < num_classes:
            sequences.append(seq)
            labels.append(label)
            prices.append(price)
    return np.array(sequences), np.array(labels, dtype=np.int64), np.array(prices)

# Check label distribution
def print_label_distribution(y, dataset_name):
    unique, counts = np.unique(y, return_counts=True)
    label_distribution = dict(zip(unique, counts))
    print(f"Label distribution in {dataset_name}: {label_distribution}")

# Dataset class
class Dataset(data.Dataset):
    def __init__(self, X, y):
        self.X, self.y = X, y

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        X = torch.tensor(self.X[idx], dtype=torch.float32).unsqueeze(0)
        y = torch.tensor(self.y[idx], dtype=torch.long)
        return X, y

# AxialLOB Model
class AxialLOB(nn.Module):
    def __init__(self, W, H, c_in, c_out, c_final, num_classes):
        super().__init__()
        self.conv1 = nn.Conv2d(1, c_in, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(c_in)
        self.conv2 = nn.Conv2d(c_in, c_out, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(c_out)
        self.conv3 = nn.Conv2d(c_out, c_final, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(c_final)
        self.pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Sequential(
            nn.Linear(c_final, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        x = torch.relu(self.bn1(self.conv1(x)))
        x = torch.relu(self.bn2(self.conv2(x)))
        x = torch.relu(self.bn3(self.conv3(x)))
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        return self.fc(x)

# Handle data imbalance using class weights
def compute_class_weights(y):
    class_sample_count = np.array([len(np.where(y == t)[0]) for t in np.unique(y)])
    weight = 1. / class_sample_count
    samples_weight = np.array([weight[t] for t in y])
    return torch.from_numpy(samples_weight).double()

# Trading Strategy Simulation
def simulate_trading_debug(preds, prices, initial_cash=10000, transaction_cost=0.001):
    cash = initial_cash
    stock = 0
    print(f"Initial cash: {cash}, transaction cost: {transaction_cost}")

    for i, (pred, price) in enumerate(zip(preds, prices)):
        if pred == 2:  # Buy signal
            num_shares = cash / (price * (1 + transaction_cost))
            cash -= num_shares * price * (1 + transaction_cost)
            stock += num_shares
            print(f"BUY at step {i}: Bought {num_shares:.2f} shares at {price:.2f}, Cash: {cash:.2f}, Stock: {stock:.2f}")
        elif pred == 0:  # Sell signal
            cash += stock * price * (1 - transaction_cost)
            print(f"SELL at step {i}: Sold {stock:.2f} shares at {price:.2f}, Cash: {cash:.2f}")
            stock = 0
        # Hold signal does nothing

    portfolio_value = cash + stock * prices[-1]
    print(f"Final Portfolio Value: {portfolio_value:.2f}, Remaining Cash: {cash:.2f}, Remaining Stock: {stock:.2f}")
    return portfolio_value

# Evaluation with Debugging
def evaluate_strategy_debug(model, train_data, test_data, horizons, batch_size=32):
    model.to(device)
    results = []
    for horizon in horizons:
        print(f"\nEvaluating for Prediction Horizon: {horizon}")
        X_train, y_train, _ = create_sequences(train_data, dim, horizon, W, num_classes)
        X_test, y_test, prices_test = create_sequences(test_data, dim, horizon, W, num_classes)

        # Print label distribution
        print_label_distribution(y_train, "training set")
        print_label_distribution(y_test, "test set")

        # Handle data imbalance
        class_weights = compute_class_weights(y_train)
        class_weights = class_weights.to(device)

        train_dataset = Dataset(X_train, y_train)
        train_sampler = data.WeightedRandomSampler(class_weights, len(class_weights))

        train_loader = data.DataLoader(train_dataset, batch_size=batch_size, sampler=train_sampler)
        test_loader = data.DataLoader(Dataset(X_test, y_test), batch_size=batch_size, shuffle=False)

        optimizer = optim.Adam(model.parameters(), lr=0.001)
        criterion = nn.CrossEntropyLoss()

        for epoch in range(10):  # Increased epochs
            model.train()
            epoch_loss = 0
            for inputs, labels in train_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                epoch_loss += loss.item()
            print(f"Epoch {epoch+1}, Loss: {epoch_loss / len(train_loader):.4f}")

        model.eval()
        all_preds = []
        with torch.no_grad():
            for inputs, _ in test_loader:
                inputs = inputs.to(device)
                outputs = model(inputs)
                preds = torch.argmax(outputs, dim=1)
                all_preds.append(preds.cpu().numpy())

        preds = np.concatenate(all_preds)
        print(f"Predictions: {preds[:10]} ... {preds[-10:]} (first 10 and last 10 predictions)")
        portfolio_value = simulate_trading_debug(preds, prices_test)
        results.append({"Horizon": horizon, "Portfolio Value": portfolio_value})

    return pd.DataFrame(results)

# Instantiate model
model = AxialLOB(W, dim, 16, 16, 16, num_classes)

# Evaluate the model
results = evaluate_strategy_debug(model, dec_train, test_data_combined, [10, 20, 30, 50, 100])
print(f"Results:\n{results}")


Using device: cpu

Evaluating for Prediction Horizon: 10
Label distribution in training set: {0: 72, 1: 23, 2: 3}
Label distribution in test set: {0: 398}
Epoch 1, Loss: 1.0942
Epoch 2, Loss: 1.0723
Epoch 3, Loss: 1.0747
Epoch 4, Loss: 1.0394
Epoch 5, Loss: 1.0468
Epoch 6, Loss: 1.0325
Epoch 7, Loss: 1.0241
Epoch 8, Loss: 1.0100
Epoch 9, Loss: 1.0265
Epoch 10, Loss: 1.0327
Predictions: [1 1 1 1 1 1 1 1 1 1] ... [1 1 1 1 1 1 1 1 1 1] (first 10 and last 10 predictions)
Initial cash: 10000, transaction cost: 0.001
SELL at step 34: Sold 0.00 shares at -0.96, Cash: 10000.00
SELL at step 35: Sold 0.00 shares at 2.40, Cash: 10000.00
SELL at step 36: Sold 0.00 shares at 0.38, Cash: 10000.00
SELL at step 37: Sold 0.00 shares at -0.04, Cash: 10000.00
SELL at step 38: Sold 0.00 shares at -0.00, Cash: 10000.00
SELL at step 39: Sold 0.00 shares at 3.36, Cash: 10000.00
SELL at step 40: Sold 0.00 shares at -0.13, Cash: 10000.00
SELL at step 41: Sold 0.00 shares at -0.06, Cash: 10000.00
SELL at step 4

C:\Users\monam\AppData\Local\Temp\ipykernel_14796\1363679894.py:124: RuntimeWarning: invalid value encountered in scalar divide
  num_shares = cash / (price * (1 + transaction_cost))


Epoch 1, Loss: 0.9757
Epoch 2, Loss: 0.8929
Epoch 3, Loss: 0.9054
Epoch 4, Loss: 0.8724
Epoch 5, Loss: 1.0063
Epoch 6, Loss: 0.8403
Epoch 7, Loss: 0.8842
Epoch 8, Loss: 0.9297
Epoch 9, Loss: 0.8517
Epoch 10, Loss: 0.8702
Predictions: [1 1 1 1 1 1 1 1 1 1] ... [1 1 1 1 1 1 1 1 1 1] (first 10 and last 10 predictions)
Initial cash: 10000, transaction cost: 0.001
SELL at step 18: Sold 0.00 shares at -0.00, Cash: 10000.00
SELL at step 19: Sold 0.00 shares at 3.36, Cash: 10000.00
SELL at step 20: Sold 0.00 shares at -0.13, Cash: 10000.00
SELL at step 21: Sold 0.00 shares at -0.06, Cash: 10000.00
SELL at step 22: Sold 0.00 shares at 0.01, Cash: 10000.00
SELL at step 23: Sold 0.00 shares at 1.50, Cash: 10000.00
SELL at step 24: Sold 0.00 shares at -1.05, Cash: 10000.00
BUY at step 25: Bought -160104.99 shares at -0.06, Cash: 0.00, Stock: -160104.99
BUY at step 26: Bought 0.00 shares at 0.01, Cash: 0.00, Stock: -160104.99
BUY at step 27: Bought 0.00 shares at 2.53, Cash: 0.00, Stock: -160104.99

In [53]:
# 11 Backtest on AAPL data from yahoo finance
import yfinance as yf
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import pandas as pd

# Set device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Fetch stock data and calculate technical indicators
def fetch_stock_data(ticker, start_date, end_date):
    df = yf.download(ticker, start=start_date, end=end_date, progress=False)
    df = df[['Open', 'High', 'Low', 'Close', 'Volume']].copy()
    
    # Add Returns
    df['Returns'] = df['Close'].pct_change()
    
    # Add Moving Average
    df['MA_10'] = df['Close'].rolling(window=10).mean()
    
    # Add Exponential Moving Average
    df['EMA_10'] = df['Close'].ewm(span=10, adjust=False).mean()
    
    # Add Relative Strength Index (RSI)
    delta = df['Close'].diff()
    gain = delta.where(delta > 0, 0).rolling(window=14).mean()
    loss = -delta.where(delta < 0, 0).rolling(window=14).mean()
    df['RSI'] = 100 - (100 / (1 + gain / loss))
    
    # Add MACD
    short_ema = df['Close'].ewm(span=12, adjust=False).mean()
    long_ema = df['Close'].ewm(span=26, adjust=False).mean()
    df['MACD'] = short_ema - long_ema
    df['Signal_Line'] = df['MACD'].ewm(span=9, adjust=False).mean()
    
    # Add Bollinger Bands
    df['MA_20'] = df['Close'].rolling(window=20).mean()
    df['BB_Upper'] = df['MA_20'] + (2 * df['Close'].rolling(window=20).std())
    df['BB_Lower'] = df['MA_20'] - (2 * df['Close'].rolling(window=20).std())
    
    # Drop NaN values
    df.dropna(inplace=True)
    
    return df

# Parameters for data fetch
ticker = "AAPL"  # Example: Apple stock
start_date = "2020-01-01"
end_date = "2023-01-01"

# Fetch data
data = fetch_stock_data(ticker, start_date, end_date)
print(f"Fetched {len(data)} rows of data.")

# Convert to numpy (include technical indicators)
data_np = data[['Open', 'High', 'Low', 'Close', 'Volume', 'Returns', 'MA_10', 'EMA_10', 'RSI', 'MACD', 'Signal_Line', 'BB_Upper', 'BB_Lower']].to_numpy()

# Parameters
W = data_np.shape[1]  # Number of features (13 in this case)
dim = 5               # Sequence length
num_classes = 3
batch_size = 32

# Create sequences
def create_sequences(data, dim, horizon):
    sequences, labels, prices = [], [], []
    N = data.shape[0]
    for i in range(N - dim - horizon + 1):
        seq = data[i:i + dim, :]
        # Label: 2 (Buy) if return > 0.001, 0 (Sell) if return < -0.001, else 1 (Hold)
        ret = data[i + dim + horizon - 1, 5]  # Returns column index
        label = 2 if ret > 0.001 else (0 if ret < -0.001 else 1)
        price = data[i + dim + horizon - 1, 3]  # Close price
        sequences.append(seq)
        labels.append(label)
        prices.append(price)
    return np.array(sequences), np.array(labels, dtype=np.int64), np.array(prices)

# Dataset class
class StockDataset(Dataset):
    def __init__(self, X, y):
        self.X, self.y = X, y

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        X = torch.tensor(self.X[idx], dtype=torch.float32).unsqueeze(0)  # Shape: (1, dim, W)
        y = torch.tensor(self.y[idx], dtype=torch.long)
        return X, y

# AxialLOB Model (Remains the same)
class AxialLOB(nn.Module):
    def __init__(self, W, H, c_in, c_out, c_final, num_classes):
        super().__init__()
        self.conv1 = nn.Conv2d(1, c_in, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(c_in)
        self.conv2 = nn.Conv2d(c_in, c_out, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(c_out)
        self.conv3 = nn.Conv2d(c_out, c_final, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(c_final)
        self.pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Sequential(
            nn.Linear(c_final, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        x = torch.relu(self.bn1(self.conv1(x)))
        x = torch.relu(self.bn2(self.conv2(x)))
        x = torch.relu(self.bn3(self.conv3(x)))
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        return self.fc(x)

# Corrected DeepLOB Model
class DeepLOB(nn.Module):
    def __init__(self, W, H, hidden_size, num_classes):
        super().__init__()
        self.W = W  # Number of features
        self.H = H  # Sequence length
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        
        # Compute LSTM input size
        self.lstm_input_size = 128 * self.W  # channels * width
        self.lstm = nn.LSTM(self.lstm_input_size, hidden_size, batch_first=True, bidirectional=True)
        
        self.fc = nn.Sequential(
            nn.Linear(2 * hidden_size, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        # x: (batch_size, 1, H, W)
        x = torch.relu(self.bn1(self.conv1(x)))  # x: (batch_size, 32, H, W)
        x = torch.relu(self.bn2(self.conv2(x)))  # x: (batch_size, 64, H, W)
        x = torch.relu(self.bn3(self.conv3(x)))  # x: (batch_size, 128, H, W)
        
        batch_size, channels, height, width = x.size()
        
        # Reshape x to (batch_size, height, channels * width) for LSTM
        x = x.permute(0, 2, 1, 3).contiguous()  # x: (batch_size, height, channels, width)
        x = x.view(batch_size, height, -1)      # x: (batch_size, height, channels * width)
        
        x, _ = self.lstm(x)                     # x: (batch_size, height, 2 * hidden_size)
        x = x[:, -1, :]                         # Take the last time step
        return self.fc(x)

# Trading Strategy Simulation (Remains the same)
def simulate_trading_debug(preds, prices, initial_cash=10000, transaction_cost=0.001):
    cash = initial_cash
    stock = 0
    for i, (pred, price) in enumerate(zip(preds, prices)):
        if price <= 0:
            continue
        if pred == 2:  # Buy
            if cash > 0:
                num_shares = cash / (price * (1 + transaction_cost))
                cash -= num_shares * price * (1 + transaction_cost)
                stock += num_shares
        elif pred == 0:  # Sell
            if stock > 0:
                cash += stock * price * (1 - transaction_cost)
                stock = 0
    portfolio_value = cash + stock * prices[-1]
    return portfolio_value

# Evaluation with Debugging (Remains the same)
def evaluate_strategy_debug(model, data_np, horizons, batch_size=32):
    results = []
    model.to(device)
    for horizon in horizons:
        print(f"\nEvaluating for Prediction Horizon: {horizon}")
        X, y, prices = create_sequences(data_np, dim, horizon)
        train_size = int(0.8 * len(X))
        X_train, X_test = X[:train_size], X[train_size:]
        y_train, y_test = y[:train_size], y[train_size:]
        prices_test = prices[train_size:]
        
        train_loader = DataLoader(StockDataset(X_train, y_train), batch_size=batch_size, shuffle=True)
        test_loader = DataLoader(StockDataset(X_test, y_test), batch_size=batch_size, shuffle=False)

        optimizer = optim.Adam(model.parameters(), lr=0.001)
        criterion = nn.CrossEntropyLoss()

        for epoch in range(10):
            model.train()
            epoch_loss = 0
            for inputs, labels in train_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                epoch_loss += loss.item()
            print(f"Epoch {epoch+1}, Loss: {epoch_loss / len(train_loader):.4f}")

        model.eval()
        all_preds = []
        with torch.no_grad():
            for inputs, _ in test_loader:
                inputs = inputs.to(device)
                outputs = model(inputs)
                preds = torch.argmax(outputs, dim=1)
                all_preds.append(preds.cpu().numpy())
        preds = np.concatenate(all_preds)
        portfolio_value = simulate_trading_debug(preds, prices_test)
        results.append({"Horizon": horizon, "Portfolio Value": portfolio_value})
    return pd.DataFrame(results)

# Instantiate models
hidden_size = 64
models = {
    "AxialLOB": AxialLOB(W, dim, 16, 16, 16, num_classes),
    "DeepLOB": DeepLOB(W, dim, hidden_size, num_classes),
}

# Evaluate each model
horizons = [5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
for model_name, model in models.items():
    print(f"\nEvaluating {model_name}")
    results = evaluate_strategy_debug(model, data_np, horizons)
    print(f"Results for {model_name}:\n{results}")


Using device: cpu
Fetched 737 rows of data.

Evaluating AxialLOB

Evaluating for Prediction Horizon: 5
Epoch 1, Loss: 1.0069
Epoch 2, Loss: 0.8917
Epoch 3, Loss: 0.8635
Epoch 4, Loss: 0.8645
Epoch 5, Loss: 0.8643
Epoch 6, Loss: 0.8829
Epoch 7, Loss: 0.8851
Epoch 8, Loss: 0.8610
Epoch 9, Loss: 0.8562
Epoch 10, Loss: 0.8541

Evaluating for Prediction Horizon: 10
Epoch 1, Loss: 0.8634
Epoch 2, Loss: 0.8631
Epoch 3, Loss: 0.8598
Epoch 4, Loss: 0.8658
Epoch 5, Loss: 0.8544
Epoch 6, Loss: 0.8531
Epoch 7, Loss: 0.8505
Epoch 8, Loss: 0.8545
Epoch 9, Loss: 0.8547
Epoch 10, Loss: 0.8539

Evaluating for Prediction Horizon: 20
Epoch 1, Loss: 0.8700
Epoch 2, Loss: 0.8618
Epoch 3, Loss: 0.8582
Epoch 4, Loss: 0.8634
Epoch 5, Loss: 0.8630
Epoch 6, Loss: 0.8585
Epoch 7, Loss: 0.8618
Epoch 8, Loss: 0.8603
Epoch 9, Loss: 0.8569
Epoch 10, Loss: 0.8552

Evaluating for Prediction Horizon: 30
Epoch 1, Loss: 0.8644
Epoch 2, Loss: 0.8767
Epoch 3, Loss: 0.8590
Epoch 4, Loss: 0.8651
Epoch 5, Loss: 0.8609
Epoch 6